# Welcome to the party

In [1]:
import numpy as np
import pandas as pd
import math
import random
import turtle
import itertools
import matplotlib.pyplot as plt

# Stuff I'm not typing in twice

In [2]:
def hand_inputs():    
    tiles_to_spots = [[1,5,9,13,8,4],[2,6,10,14,9,5],[3,7,11,15,10,6],[11,16,21,26,20,15],[21,27,33,38,32,26],#1-5
        [32,38,43,47,42,37],[42,47,51,54,50,46],[41,46,50,53,49,45],[40,45,49,52,48,44],[29,35,40,44,39,34],#6-10
        [17,23,29,34,28,22],[8,13,18,23,17,12],[9,14,19,24,18,13],[10,15,20,25,19,14],[20,26,32,37,31,25],#11-15
        [31,37,42,46,41,36],[30,36,41,45,40,35],[18,24,30,35,29,23],[19,25,31,36,30,24]]#16-19
    tiles_to_tr = [[21,27,22,14,10,13],[38,44,39,31,27,30],[55,61,56,48,44,47],[64,68,65,57,53,56],[70,72,71,66,62,65],#1-5
            [66,69,67,59,54,58],[59,63,60,52,46,51],[42,46,43,35,29,34],[25,29,26,18,12,17],[16,20,17,9,5,8],#6-10
            [7,11,8,3,1,2],[14,19,15,7,4,6],[31,36,32,23,19,22],[48,53,49,40,36,39],[57,62,58,50,45,49],#11-15
            [50,54,51,42,37,41],[33,37,34,25,20,24],[23,28,24,16,11,15],[40,45,41,33,28,32]]#16-19
    spots_to_tiles = [[1],[2],[3],[1],[1,2],[2,3],#1-6
    [3],[1,12],[1,2,13],[2,3,14],[3,4],[12],#7-12
    [1,12,13],[2,13,14],[3,14,4],[4],[12,11],[12,13,18],#13-18
    [13,14,19],[14,4,15],[4,5],[11],[12,11,18],[13,18,19],#19-24
    [14,19,15],[4,15,5],[5],[11],[11,18,10],[18,19,17],#25-30
    [19,15,16],[15,5,6],[5],[11,10],[18,10,17],[19,17,16],#31-36
    [15,16,6],[5,6],[10],[10,17,9],[17,16,8],[16,6,7],#37-42
    [6],[10,9],[9,17,8],[16,8,7],[6,7],[9],#43-48
    [9,8],[8,7],[7],[9],[8],[7]]#49-54
    spots_to_tr=[[13,21],[30,38],[47,55],[10,13],[21,27,30],[38,44,47],[55,61],[6,10,14],[22,27,31],#1-9
                 [39,44,48],[56,61,64],[4,6],[14,19,22],[31,36,39],[48,53,56],[64,68],[2,4,7],[15,19,23],#10-18
                 [32,36,40],[49,53,57],[65,68,70],[1,2],[7,11,15],[23,28,32],[40,45,49],[57,62,65],[70,72],#19-27
                 [1,3],[8,11,16],[24,28,33],[41,45,50],[58,62,66],[71,72],[3,5,8],[16,20,24],[33,37,41],#28-36
                 [50,54,58],[66,69,71],[5,9],[17,20,25],[34,37,42],[51,54,59],[67,69],[9,12,17],[25,29,34],#37-45
                 [42,46,51],[59,63,67],[12,18],[26,29,35],[43,46,52],[60,63],[18,26],[35,43],[52,60]]#46-54
    def make_tr_to_spots(hard_writen):
        result=[]
        for i in range(1,73):
            temp=[]
            for x in range(1,55):
                if hard_writen[x-1].count(i)==1: temp.append(x)
            result.append(temp)
        return result
    def make_tr_connections(tr_to_s,s_to_tr):
        tr_connections = []
        for tr in range(len(tr_to_s)):
            touch=[]
            for s in tr_to_s[tr]:
                touch += s_to_tr[s-1]
                touch.remove(tr+1)
            tr_connections.append(touch)
        return tr_connections    
    tr_to_spots = make_tr_to_spots(spots_to_tr)
    tr_connections = make_tr_connections(tr_to_spots,spots_to_tr)
    
    return tiles_to_spots, spots_to_tiles, tiles_to_tr, spots_to_tr, tr_to_spots, tr_connections

# Class for board

In [3]:
class board:

    def __init__(self):
        self.bytile=0
        self.byspot=0
        self.spot_weight_all=0
        self.spot_weight_run=0
        self.port_info=0
        self.tiles_to_spots, self.spots_to_tiles, self.tiles_to_tr,self.spots_to_tr, self.tr_to_spots, self.tr_connections = hand_inputs()
        self.s_to_tr=pd.DataFrame({'tr':self.spots_to_tr},index=range(1,55))
        self.tr_to_s=pd.DataFrame({'spot':self.tr_to_spots},index=range(1,73))
        self.disp=0
        self.hcs=0
        
    def fill(self):
        self.set_board() ; self.make_port_info() ; self.spot_org() ; self.produce_spot_weights()
        return
    def set_board(self): #randomly sets board in reference to the tile
        recs = ["wood"]*4+["sheep"]*4+["wheat"]*4+["brick"]*3+["ore"]*3+["desert"]
        recs = random.sample(recs,19)
        raw_tokens = [10,2,9,10,8,5,11,6,5,8,9,12,6,4,3,4,3,11]
        raw_dots = [3,1,4,3,5,4,2,5,4,5,4,1,5,3,2,3,2,2]
        cols=['resources','token_numbers','token_dots','spots','roads']
        bytile = pd.DataFrame(columns=cols); j=0
        for i in range(19):
            a=recs[i]; d=self.tiles_to_spots[i] ;e=self.tiles_to_tr[i]
            if a!='desert': b=raw_tokens[j] ;c=raw_dots[j]; j+=1
            else: b=0; c=0
            new=pd.DataFrame([[a,b,c,d,e]],index=[i+1],columns=cols)
            bytile = pd.concat([bytile,new],ignore_index=False)
        self.bytile = bytile
        return
    def spot_org(self): #uses the tile-oriented 'board' to organize in reference to spots
        spots_to_tiles = self.spots_to_tiles ; board=self.bytile
        spot_resources=[]; spot_tokens=[]; spot_dots=[]; amount_tokens=[]; amount_dots=[]
        for i in spots_to_tiles:
            resources_list=[]; tokens_list=[]; dots_list=[]
            for x in i:
                resources_list.append(board['resources'][x])
                tokens_list.append(board['token_numbers'][x])
                dots_list.append(board['token_dots'][x])
            spot_resources.append(resources_list)
            spot_tokens.append(tokens_list)
            spot_dots.append(dots_list)
            amount_tokens.append(len(tokens_list))
            amount_dots.append(sum(dots_list))
        ports = list(np.zeros(54))
        for i in self.port_info.index:
            temp_spots=self.port_info.loc[i,'spots']
            temp_port=self.port_info.loc[i,'port']
            for j in temp_spots:
                ports[j-1] = temp_port
        self.byspot=pd.DataFrame({'spots_to_tiles':self.spots_to_tiles, 'spot_resources':spot_resources,
            'spot_tokens':spot_tokens,'spot_dots':spot_dots,'spot_trs':self.s_to_tr.tr,
           'amount_tokens':amount_tokens,'amount_dots':amount_dots,'ports':ports},index=range(1,55))
        return
    def make_port_info(self):
        spots = [[12,17],[4,1],[2,6],[11,16],[27,33],[43,47],[53,50],[48,52],[34,39]]
        p = ['2:1wood','2:1sheep','2:1wheat','2:1brick','2:1ore']+['3:1']*4 ; port = random.sample(p,9)
        self.port_info = pd.DataFrame({'port':port,'spots':spots})
        return
    def produce_spot_weights(self):
        spot_set=self.byspot ; port_info=self.port_info
        threes=list(np.zeros(54)); wood2=list(np.zeros(54)); sheep2=list(np.zeros(54))
        wheat2=list(np.zeros(54)); brick2=list(np.zeros(54)); ore2=list(np.zeros(54))
        for row in self.port_info.index:
            if port_info.loc[row,'port']=='3:1':
                for i in port_info.loc[row,'spots']: threes[i-1]=1
            elif port_info.loc[row,'port'][3:]=='wood':
                for i in port_info.loc[row,'spots']: wood2[i-1]=1
            elif port_info.loc[row,'port'][3:]=='sheep':
                for i in port_info.loc[row,'spots']: sheep2[i-1]=1
            elif port_info.loc[row,'port'][3:]=='wheat':
                for i in port_info.loc[row,'spots']: wheat2[i-1]=1
            elif port_info.loc[row,'port'][3:]=='brick':
                for i in port_info.loc[row,'spots']: brick2[i-1]=1
            elif port_info.loc[row,'port'][3:]=='ore':
                for i in port_info.loc[row,'spots']: ore2[i-1]=1
        wood=np.zeros(54) ; sheep=np.zeros(54) ; wheat=np.zeros(54) ; brick=np.zeros(54) ; ore=np.zeros(54)
        for i in range(54):
            run=spot_set.loc[i+1,'spot_resources'] ; gun=spot_set.loc[i+1,'spot_dots']
            for x in range(len(run)):
                if run[x]=='wood': wood[i]=gun[x]
                if run[x]=='sheep': sheep[i]=gun[x]
                if run[x]=='wheat': wheat[i]=gun[x]
                if run[x]=='brick': brick[i]=gun[x]
                if run[x]=='ore': ore[i]=gun[x]
        wood=list(wood); sheep=list(sheep); wheat=list(wheat); brick=list(brick); ore=list(ore)
        self.spot_weight_all=pd.DataFrame({'wood':wood,'sheep':sheep,'wheat':wheat,'brick':brick,'ore':ore,
            'dots':spot_set['amount_dots'],'3:1':threes,
            '2:1wood':wood2,'2:1sheep':sheep2,'2:1wheat':wheat2,'2:1brick':brick2,'2:1ore':ore2},index=range(1,55))
        self.spot_weight_run=pd.DataFrame({'wood':wood,'sheep':sheep,'wheat':wheat,'brick':brick,'ore':ore,
            'dots':spot_set['amount_dots'],'3:1':threes,
            '2:1wood':wood2,'2:1sheep':sheep2,'2:1wheat':wheat2,'2:1brick':brick2,'2:1ore':ore2},index=range(1,55))
        return
    def branching(self, cpu, max_distance, min_distance, start, raw=False):
        if raw:
            s_to_tr = pd.DataFrame({'tr':self.spots_to_tr},index=range(1,55))
            tr_to_s = pd.DataFrame({'spot':self.tr_to_spots},index=range(1,73))
        else: s_to_tr = self.s_to_tr ; tr_to_s = self.tr_to_s
        ###pick prospect spot
        ss_keep=[[start]] ; ss_long=[start] ; tr_keep=[[0]] ; rc_keep=[]
        for k in range(max_distance): # for each tier of road distance from start
            tempss_list=[] ; temptr_list=[] ; temprc_list=[] ; rc=0
            for i in ss_keep[k]: # for each stemming spot
                roads = s_to_tr.loc[i,'tr']
                for j in roads: #for each road on stemming spot
                    for spot in tr_to_s.loc[j,'spot']: # for each spot on road
                        if ss_long.count(spot)==0: # if the list of all spots seen hasn't seen this spot
                            tempss_list.append(spot) ; ss_long.append(spot) ; temptr_list.append(j) ; rc+=1
                temprc_list.append(rc)            
            ss_keep.append(tempss_list) ; tr_keep.append(temptr_list) ; rc_keep.append(temprc_list)
        prospects = pd.DataFrame(columns=['spot','roads','distance','start']) ; index=0
        for i in range(len(ss_keep)): # for every tier of distance away
            for j in range(len(ss_keep[i])): # for every spot within distance tier
                spot=ss_keep[i][j] ; road=[tr_keep[i][j]] ; flag=j
                if i>0: #if the distance tier is atleast 1 road away
                    for k in range(i-1,-1,-1): # reverse order of every tier of distance away starting at current i distance
                        temp=np.array(rc_keep[k]) ; token = rc_keep[k].index(min(temp[temp>=flag+1]))
                        road.append(tr_keep[k][token]) ; flag=token
                    road.reverse() ; road.remove(0) ; road = [road]
                new_row = pd.DataFrame({'spot':spot,'roads':road,'distance':i,'start':start},index=[index]) ; index+=1
                if list(self.spot_weight_run.index).count(spot)==1:
                    prospects = pd.concat([prospects,new_row],ignore_index=True)
        smallest=[] ; make_shift =  pd.DataFrame(data=[[[]]*4],columns=['spot','roads','distance','start'])
        for small in range(max_distance,min_distance-1,-1):
            if list(prospects.distance).count(small)>0: smallest=small
        if smallest==[]: return make_shift , make_shift, []
        prospects = prospects.tail(len(prospects)-(list(prospects.distance).index(smallest)))
        tossers=[]
        for i in prospects.spot:
            if list(self.spot_weight_run.index).count(i)==0: tossers.append(i)
        if len(tossers)==len(list(prospects.spot)): return make_shift, make_shift, []
        for i in tossers: prospects = prospects[prospects.spot!=i]
        prospects =pd.concat([prospects,pd.DataFrame(columns=['spot','roads','distance','start'])],ignore_index=True)
        calculations = np.dot(self.spot_weight_run.loc[prospects.spot,:],cpu.value.T)
        chosen_prospect_index = np.argmax(calculations)
        scores = pd.DataFrame({'score':list(calculations),'spot':list(prospects.spot)})
        choice = prospects.loc[chosen_prospect_index,:]
        return choice, prospects, scores
    
    def update_running_weights(self,cpu, settlement, roads, city):
        #update self.spot_weight_run
        claimed_spots = settlement + city ; claimed_roads = roads
        for spot in claimed_spots:
            _, too_close, _ = self.branching(cpu,1,1,spot,raw=True)
            keepers_w = list(self.spot_weight_run.index)
            if keepers_w.count(spot)>0: keepers_w.remove(spot)
            for i in too_close.spot:
                if keepers_w.count(i)>0: keepers_w.remove(i)
            self.spot_weight_run = self.spot_weight_run.loc[keepers_w,:]
        #update self.s_to_tr
            #keepers_ss = list(self.s_to_tr.index) ; keepers_ss.remove(spot) #keep ss
            #self.s_to_tr = self.s_to_tr.loc[keepers_ss,:]
            for i in self.tr_to_s.index:
                bag=[]
                for j in self.tr_to_s.loc[i,'spot']:
                    if j!=spot: bag.append(j)
                self.tr_to_s.loc[i,'spot'] = bag
        #update self.tr_to_s
        for tr in claimed_roads:
            #keepers_trs = list(self.tr_to_s.index) ; keepers_trs.remove(tr)
            #self.tr_to_s = self.tr_to_s.loc[keepers_trs,:]
            for i in self.s_to_tr.index:
                bag=[]
                for j in self.s_to_tr.loc[i,'tr']:
                    if j!=tr: bag.append(j)
                self.s_to_tr.loc[i,'tr'] = bag    
        return
    def display_board(self,event):
        turtle.title('Welcome to Catan')
        cols = ['red','white','blue','orange']; cn = ['rc','rt','rd','rs']
        rcs = {'wood':'dark green','sheep':'green yellow','wheat':'gold',
        'brick':'chocolate','ore':'slate grey','desert':'khaki'}
        disp = pd.DataFrame(columns=cn)
        for i in range(1,20):
            rc = rcs[self.bytile.loc[i,'resources']]
            rt = self.bytile.loc[i,'token_numbers']
            rd = self.bytile.loc[i,'token_dots']
            rt_temp = ''
            rs = self.bytile.loc[i,'spots']
            for j in range(rd): rt_temp += '.'
            rd = rt_temp
            temp = pd.DataFrame(data=np.array([[rc,rt,rd,rs]]),columns=cn)
            disp = pd.concat([disp,temp],ignore_index=True)
        stretch=45; hcs = []
        th = round(math.sin(30*math.pi/180)*stretch,5)
        h = stretch + th*2; up = stretch+th
        w = round(math.sin(120*math.pi/180)*stretch/math.sin(30*math.pi/180),5)
        places = [(-w,up*2),(0,up*2),(w,up*2),(1.5*w,up),(2*w,0),(1.5*w,-up),(w,-up*2),(0,-up*2),(-w,-up*2),
           (-1.5*w,-up),(-2*w,0),(-1.5*w,up),(-0.5*w,up),(0.5*w,up),(w,0),(0.5*w,-up),(-0.5*w,-up),(-w,0),(0,0)]    
        hex_dim = ((h/2,0),(stretch/2,w/2),(-stretch/2,w/2),(-h/2,0),(-stretch/2,-w/2),(stretch/2,-w/2))
        turtle.clearscreen()
        author = turtle.Turtle(visible=False)
        author.penup(); author.speed(0)
        author.screen.bgcolor('cornflower blue')
        crop = turtle.Shape('compound')
        def add_labels(stretch, t,disp,th, n):
            font=('Courier',int(stretch*0.55), 'bold')
            t.write(disp.rt[n],font=font,align='center')
            (x,y) = t.position()
            y-=15; t.setposition((x,y))
            t.write(disp.rd[n],font=font,align='center')
            return
        for i in range(19):
            x=places[i][0]; y=places[i][1]
            crop.addcomponent(np.add(hex_dim,[-y,x]),disp.rc[i],"black")
        turtle.register_shape("crop", crop)
        author.shape('crop') ; author.stamp(); author.hideturtle()
        for i in range(19): x=places[i][0]; y=places[i][1]; author.setpos((x,y)); add_labels(stretch,author,disp,th,i)
        px= [-2.25,-1.35,0.31,1.85,2.75,1.9,0.35,-1.35,-2.25]; py= [0.75,1.9,1.9,1.15,0,-1.3,-2.05,-2.10,-0.8]
        for port in range(9):
            author.setpos(px[port]*w,py[port]*h); rec = self.port_info.port[port][3:]
            if rec=='': author.color('black')
            else: author.color(rcs[rec])
            scrib=self.port_info.port[port][:3]; font=('Courier', int(stretch*0.26), 'bold')
            author.write(scrib,False,font=font,align='center')
        self.hcs=places ; self.disp=disp
        return
    def display_add_builds(self, cpu, settlements, cities, roads):
        stretch = 45 ; w = round(math.sin(120*math.pi/180)*stretch/math.sin(30*math.pi/180),5)
        bob = turtle.Turtle(visible=False) ; bob.speed(0); bob.pensize(7); bob.penup()
        t_s = self.tiles_to_spots ; t_tr = self.tiles_to_tr
        cols=['tile','order','build','shape']; builds = pd.DataFrame(columns=cols)
        for i in settlements+cities:
            more=True ; j=-1
            while more:
                j+=1; rack=t_s[j]
                if len(list(set([i]) & set(rack)))>0:
                    more=False; order=rack.index(i)
                    if settlements.count(i)>0: build='s'; shape='circle'
                    else: build='c'; shape='square'
                    new=pd.DataFrame([[j,order,build,shape]],columns=cols); builds = pd.concat([builds,new],ignore_index=True)
        for i in roads:
            more=True;j=-1
            while more:
                j+=1; rack=t_tr[j]
                if len(list(set([i]) & set(rack)))>0:
                    more=False; order=rack.index(i)
                    new=pd.DataFrame([[j,order,'r','turtle']],columns=cols); builds = pd.concat([builds,new],ignore_index=True)
        adjust=5
        for idx in builds.index:
            deg=60*builds.order[idx]; rad=deg*math.pi/180
            a=math.sin(rad)*(stretch); b=math.cos(rad)*stretch
            (x,y) = self.hcs[builds.tile[idx]]
            x2=x+a ; y2=y+b ; bob.setposition((x2,y2))
            bob.shape(builds.loc[idx,'shape']); bob.color(cpu.color)
            if builds.build[idx]=='r':
                deg=60*builds.order[idx]+20; rad=deg*math.pi/180
                a=math.sin(rad)*(stretch-adjust); b=math.cos(rad)*(stretch-adjust)
                x2=x+a; y2=y+b ; bob.setposition(x2,y2); bob.pendown()
                deg=60*builds.order[idx]+40; rad=deg*math.pi/180
                a=math.sin(rad)*(stretch-adjust); b=math.cos(rad)*(stretch-adjust)
                x2=x+a; y2=y+b ; bob.setposition(x2,y2); bob.penup()
            else:
                bob.stamp() ; y2-=10 ; bob.setposition(x2,y2)
                bob.color('black')
                bob.write(builds.build[idx],font=('Courier', 15, 'italic'),align='center')
        return

# Class for players

In [4]:
class player:
    
    def __init__(self,who_is_it,color):
        self.name = who_is_it #their name
        self.color = color
        desire_names=['road','settlement','city','dcard']
        r=random.sample(range(0,101,1),1)[0]/100
        s=random.sample(range(100,301,1),1)[0]/100
        c=random.sample(range(100,301,1),1)[0]/100
        d=random.sample(range(0,101,1),1)[0]/100
        desire=[r,s,c,d]
        trait_names=['trade_ins','diversity','optimism','long_term_thinker','ibs','ttc','pg']
        ltt = random.sample([1,2,3,4],1)[0]
        ti = random.sample(range(0,501,1),1)[0]/100
        ibs = random.sample(range(0,501,1),1)[0]/100
        ttc = random.sample(range(0,501,1),1)[0]/100
        div = random.sample(range(0,501,1),1)[0]/100
        pg = random.sample(range(0,501,1),1)[0]/100
        trait=[ti,div,1.2, ltt, ibs, ttc, pg]        
        value_names=['wood','sheep','wheat','brick','ore','dots','3:1',
                     '2:1wood','2:1sheep','2:1wheat','2:1brick','2:1ore']
        value=[0,0,0,0,0,0,ti,0,0,0,0,0]
        #strategy dataframes
        self.desire = pd.DataFrame(np.array([desire]),columns=desire_names)
        #self.desire.loc[0,:] = np.true_divide(random.sample(range(0,201,1),4),100)
        self.value = pd.DataFrame(np.array([value]),columns=value_names)
        self.trait = pd.DataFrame(np.array([trait]),columns=trait_names)
        self.dcards = []
        

# Class for gameplay

In [5]:
class game:
    
    def __init__(self,players):
        #excavate and solve
        self.fix=None
        #data
        self.chosen_buy = pd.DataFrame(columns=['item','location','score','turns_away','turn'])
        self.card_data = pd.DataFrame(columns=['wood','sheep','wheat','brick','ore','total','time','turn'])
        self.status_data = pd.DataFrame(columns=['status','turn'])
        self.placements = 0
        #turtles
        self.robber_display=0
        self.status = 'none'
        self.status_man = 0
        self.status_switch = 'off'
        self.hist = turtle.Turtle() ; self.card_stamp = 0
        self.author=turtle.Turtle() ; self.author.speed(0); self.author.hideturtle() ; self.author.penup()
        self.z=turtle.Turtle() ; self.z.speed(0); self.z.hideturtle() ; self.z.penup()
        self.roll_man = turtle.Turtle(visible=False) ; self.roll_man.penup()
        self.roll_man.speed(0) ; self.roll_man.setposition(0,-275)
        self.trade_turtle = turtle.Turtle(visible=False) ; self.trade_turtle.penup() ; self.trade_turtle.speed(0)
        self.trade_turtle.setpos(-190,200)
        self.lr = turtle.Turtle(visible=False); self.lr.penup(); self.lr.speed(0); self.lr.setpos(235,130)
        self.la = turtle.Turtle(visible=False); self.la.penup(); self.la.speed(0); self.la.setpos(235,150)
        self.factor_horizontal=1.5 ; self.factor_verticle=25
        self.vp_man = turtle.Turtle(visible=False); self.vp_man.penup(); self.vp_man.speed(0)
        #players and placement_order
        idx = random.sample(range(len(players)),len(players))
        self.players = pd.DataFrame(columns=['player'])
        self.names = [[]]*len(players)
        tick=0
        for i in idx:
            self.names[tick] = players[i].name
            new=pd.DataFrame(players[i],index=[self.names[tick]],columns=['player'])
            self.players = pd.concat([self.players,new],ignore_index=False)
            tick+=1
        names=self.names
        idx = len(self.names) ; order = []
        for i in range(0,idx,1): order += [self.names[i]]
        for i in range(idx-1,-1,-1): order += [self.names[i]]
        self.placement_order = order
        #explanatory variables
        cols=['road_des','set_des','city_des','dcard_des','ti','ltt','div','ibs','ttc']
        explan=pd.DataFrame(columns=cols)
        for cpu in self.players.player:
            rd=cpu.desire.road[0]; sd=cpu.desire.settlement[0]; cd=cpu.desire.city[0]; dd=cpu.desire.dcard[0]
            ti=cpu.trait.trade_ins[0]; ltt=cpu.trait.long_term_thinker[0]; div=cpu.trait.diversity[0]
            ibs=cpu.trait.ibs[0]; ttc=cpu.trait.ttc[0]
            temp = pd.DataFrame([[rd,sd,cd,dd,ti,ltt,div,ibs,ttc]],index=[cpu.name],columns=cols)
            explan = pd.concat([explan,temp],ignore_index=False)
        self.explanatory = explan
        # response variables
        self.response = 0
        #place
        place = board() ; place.fill()
        self.place=place
        #rolls
        self.rolls=[]
        self.robber=list(self.place.bytile[self.place.bytile.resources=='desert'].index)[0]
        #dcards
        dcards=(['knight']*14+['road_builder']*6+['year_of_plenty']*6+['monopoly']*6+['victory_point']*5)
        self.dcards = random.sample(dcards,len(dcards))
        self.knights = pd.DataFrame(0,index=names,columns=['knights'])
        self.longest_road = 5
        self.dcard_play = pd.DataFrame(columns=['dcard','turn'])
        #make trades
        self.trade_hand = 0
        self.trade_play = 0
        self.trade_value = 0
        #posessions and attributes
        self.topspot=0
        pos_names = ['settlements','roads','cities','spots_blocked','roads_blocked','proxy_spots']
        card_names = ['wood','sheep','wheat','brick','ore','total']
        point_names = ['cities','settlements','longest_road','largest_army','victory_point','total']
        trade_in_names = ['wood','sheep','wheat','brick','ore']
        self.pos=pd.DataFrame([[[]]*len(pos_names)]*len(names),index=names,columns=pos_names)
        self.cards = pd.DataFrame(0,index=names,columns=card_names)
        self.alltime_cards = pd.DataFrame(0,index=names,columns=card_names)
        self.points = pd.DataFrame(0,index=names,columns=point_names)
        self.trade_ins = pd.DataFrame(4,index=names,columns=trade_in_names)
        self.probs = pd.DataFrame(0,index=names,columns=card_names[:5])
        items = ['road','settlement','city','dcard'] ; prices=[[1,0,0,1,0],[1,1,1,1,0],[0,0,2,0,3],[0,1,1,0,1]]
        self.buy_ref = pd.DataFrame(data=prices,index=items,columns=list(self.trade_ins.columns))
    def update_pos(self, cpu, settlement, roads, city):
        self.status = 'update_pos' ; self.display_status()
        for name in self.names:
            if name==cpu.name:
                self.pos.loc[name,'settlements'] = self.pos.loc[name,'settlements']+settlement
                self.pos.loc[name,'roads'] = self.pos.loc[name,'roads']+roads
                self.pos.loc[name,'cities'] = self.pos.loc[name,'cities']+city
                if len(list(city))>0:
                    pull = list(self.pos.loc[name,'settlements'])
                    pull.remove(list(city)[0])
                    self.pos.loc[name,'settlements'] = pull
            else:
                self.pos.loc[name,'spots_blocked'] = self.pos.loc[name,'spots_blocked']+settlement
                self.pos.loc[name,'roads_blocked'] = self.pos.loc[name,'roads_blocked']+roads
            if list(self.pos.loc[name,'proxy_spots']).count([settlement])>0:
                a=list(self.pos.loc[name,'proxy_spots']) ; b=[]
                t = a.index(settlement)
                for i in range(len(a)):
                    if i!=t: b+=[a[i]]
                self.pos.loc[name,'proxy_spots'] = b
        proxy=[]
        cross_remove = self.pos.loc[cpu.name,'settlements'] + self.pos.loc[cpu.name,'cities'] + self.pos.loc[cpu.name,'proxy_spots']
        for i in roads: proxy = proxy + self.place.tr_to_s.loc[i,'spot']
        proxy=list(set(proxy))
        for i in cross_remove:
            amount=proxy.count(i)
            if amount>0:
                for j in range(amount): proxy.remove(i)
        self.pos.loc[cpu.name,'proxy_spots'] = self.pos.loc[cpu.name,'proxy_spots']+proxy
        #make updates to dataframes contingent on the self.pos dataframe
        self.update_trade_ins(cpu)
        self.update_probs(cpu)
        self.update_points(cpu)
        self.update_values(cpu,settlement,city)
        if len(roads)>0: self.update_longest_road(cpu)
        return
    def update_trade_ins(self,cpu):
        self.status = 'update_trade_ins' ; self.display_status()
        properties = self.pos.loc[cpu.name,'cities'] + self.pos.loc[cpu.name,'settlements']
        cols = self.trade_ins.columns
        for i in properties:
            port = self.place.byspot.loc[[i],'ports']
            port = list(port)[0]
            if port !=0:
                if port=='3:1':
                    for resource in cols: self.trade_ins.loc[cpu.name,resource] = 3
                else: self.trade_ins.loc[cpu.name,port[3:]] = 2
        return
    def update_probs(self,cpu):
        self.status = 'update_probs' ; self.display_status()
        self.probs.loc[cpu.name,:] = [0,0,0,0,0]
        settlements = self.pos.loc[cpu.name,'settlements'] ; cities = self.pos.loc[cpu.name,'cities']
        for spot in settlements:
            resources = self.place.byspot.loc[spot,'spot_resources'] ; dots = self.place.byspot.loc[spot,'spot_dots']
            for i in range(len(resources)):
                if resources[i]!='desert': self.probs.loc[cpu.name,resources[i]] = round(self.probs.loc[cpu.name,resources[i]] + dots[i]/36,3)
        for spot in cities:
            resources = self.place.byspot.loc[spot,'spot_resources'] ; dots = self.place.byspot.loc[spot,'spot_dots']   
            for i in range(len(resources)):
                if resources[i]!='desert': self.probs.loc[cpu.name,resources[i]] = round(self.probs.loc[cpu.name,resources[i]] + dots[i]/18,3)
        return
    def update_points(self,cpu):
        self.status = 'update_points' ; self.display_status()
        settlements_points = len(self.pos.loc[cpu.name,'settlements'])
        cities_points = len(self.pos.loc[cpu.name,'cities'])*2
        self.points.loc[cpu.name,'settlements'] = settlements_points
        self.points.loc[cpu.name,'cities'] = cities_points
        #longest_road, largest_army, victory_points not accounted for
        cols = len(self.points.columns) ; total_points = sum(self.points.loc[cpu.name,:][:cols-1]) 
        self.points.loc[cpu.name,'total'] = total_points
        return
    def make_placements(self):
        self.place.display_board(self)
        self.la.write('Largest Army',font=('Courier',20,'bold'),align='center')
        self.lr.write('Longest Road',font=('Courier',20,'bold'),align='center')
        ### robber turtle
        self.robber_display = turtle.Turtle()
        self.robber_display.shape('circle')
        self.robber_display.color('silver')
        self.robber_display.penup()
        self.robber_display.speed(0)
        self.robber_display.setposition(self.place.hcs[self.robber-1])
        ### status turtle
        self.status_man = turtle.Turtle() ; self.status_man.speed(0)
        self.status_man.penup() ; self.status_man.hideturtle() ; self.status_man.setposition(0,-250)
        ### making placements
        for name in self.placement_order:
            cpu = list(self.players.loc[name,:])[0]
            start = np.argmax(np.dot(self.place.spot_weight_run,cpu.value.T))
            start = list(self.place.spot_weight_run.index)[start]
            pick, _, _ = self.place.branching(cpu,3,2,start)
            if pick.start!=[[]]: settlement = [pick.start] ; roads = [pick.roads[0]] ; city = []
            else: settlement=[start]; roads=[]; city=[]
            self.update_pos(cpu, settlement, roads, city)
            self.place.update_running_weights(cpu, settlement, roads, city)
            self.place.display_add_builds(cpu, settlement, city, roads)
        for cpu in self.players.player:
            for i in self.pos.loc[cpu.name,'settlements']:
                for j in self.place.byspot.loc[i,'spot_resources']:
                    if j!='desert':
                        self.cards.loc[cpu.name,j] = self.cards.loc[cpu.name,j] + 1
                        self.alltime_cards.loc[cpu.name,j] = self.alltime_cards.loc[cpu.name,j] + 1
            self.cards.loc[cpu.name,'total'] = sum(self.cards.loc[cpu.name,:][:5])
            self.alltime_cards.loc[cpu.name,'total'] = sum(self.alltime_cards.loc[cpu.name,:][:5])
        self.placements = self.pos
        self.display_cards()
        return
    def roll_dice(self):
        roll = random.sample(range(1,7),1)[0] + random.sample(range(1,7),1)[0]
        return roll
    def collect_cards(self):
        roll = self.roll_dice()
        self.status = 'collect_cards-'+str(roll) ; self.display_status()
        self.rolls.append(roll)
        self.roll_man.clear()
        self.roll_man.write('rolled #'+str(roll),font=('Courier',20, 'bold'),align='center')
        name = self.names[len(self.rolls) % len(self.names)-1] ; cpu = self.players.player[name]
        self.collect_card_data(cpu,'start')
        if roll==7: self.drop_cards() ; victim = self.move_robber(cpu) ; self.draw_card(cpu,victim)
        for cpu in self.players.player:
            settlements = self.pos.loc[cpu.name,'settlements'] ; cities = self.pos.loc[cpu.name,'cities']
            spots = settlements+cities
            for i in spots:
                factor=1
                if cities.count(i)>0: factor=2
                recs = self.place.byspot.spot_resources[i] ; tokens = self.place.byspot.spot_tokens[i]
                tiles = self.place.byspot.spots_to_tiles[i]
                for j in range(len(tokens)):
                    if tokens[j]==roll and tiles[j] != self.robber:
                        self.cards.loc[cpu.name,recs[j]] =  self.cards.loc[cpu.name,recs[j]] + 1*factor
                        self.alltime_cards.loc[cpu.name,recs[j]] =  self.alltime_cards.loc[cpu.name,recs[j]] + 1*factor
            self.cards.loc[cpu.name,'total'] =  sum(self.cards.loc[cpu.name,:][:5])
            self.alltime_cards.loc[cpu.name,'total'] =  sum(self.alltime_cards.loc[cpu.name,:][:5])
        name = self.names[len(self.rolls) % len(self.names)-1] ; cpu = self.players.player[name]
        self.collect_card_data(cpu,'collect_cards')
        self.play_dcards(cpu)
        return
    def all_possible_hands(self, cpu, turns_away):
        self.status = 'all_possible_hands' ; self.display_status()
        if turns_away>0: cards_held = self.predict_hand(cpu)
        else: cards_held = list(self.cards.loc[cpu.name,:])[:5]
        resources = list(self.cards.columns)[:5]
        trade_rate = list(self.trade_ins.loc[cpu.name,:]) ; liquid = []
        [liquid.append(cards_held[i] // trade_rate[i]) for i in range(len(resources))]
        if sum(liquid)==0: return pd.DataFrame([self.cards.loc[cpu.name,:][:5]],index=[0])
        truth=[]; hatch=[]
        for i in range(5): truth+=[i]*liquid[i]
        if sum(truth)>0: loop=range(1,sum(truth)+1)
        else: loop=[1]
        for i in loop:
            hatch+=list(itertools.combinations(truth,i))
        if type(hatch)==tuple: hatch = list(pd.DataFrame(np.array([hatch]).T)[0].unique())
        elif type(hatch)==list: hatch = list(pd.DataFrame(np.array(hatch).T)[0].unique())
        if type(hatch[0])==int: loop2=[hatch]
        else: loop2=hatch
        diffs=[] ; tots=[]
        for i in loop2:
            temp=[0]*5; temp_tots=[0]*5
            if type(i)!=list and type(i)!=tuple: i=[i]
            for j in i:
                temp_tots[j] = list(i).count(j)
                temp[j] = list(i).count(j)*trade_rate[j]
            diffs.append(temp)
            tots.append(sum(temp_tots))
        temp=[]
        for k in range(len(tots)):
            good=[]; j=0
            for i in diffs[k]:
                if i==0: good+=[j]
                j+=1
            possible = list(set([0,1,2,3,4]) & set(good))
            i=tots[k]; j=list(np.subtract(cards_held,diffs[k]))
            a=list(itertools.combinations(possible*i,i))
            a=list(pd.DataFrame([a]).T[0].unique())
            for i in a:
                indv=[0]*5
                for m in i: indv[m]+=1
                new=tuple(np.add(j,indv))
                if (new[0]>0 and new[3]>0) or (new[2]>1 and new[4]>2) or (new[1]>0 and new[2]>0 and new[4]>0):
                    temp.append(new)
        temp = list(pd.DataFrame([temp]).T[0].unique())
        temp = pd.DataFrame(temp,columns=resources)
        #orig=pd.DataFrame([cards_held],index=[0])
        orig = pd.DataFrame([self.cards.loc[cpu.name,:][:5]],index=[0])
        temp = pd.concat([orig,temp],ignore_index=True)
        return temp
    def hand_option_variants(self,dif,hand):
        #self.status = 'hand_option_variants' ; self.display_status()
        buy_ref = self.buy_ref ; items_list = buy_ref.index ; rec_list = buy_ref.columns
        ovs = pd.DataFrame(0,index=[0],columns=items_list) #option variants -> ovs
        items_amount = pd.DataFrame(0,index=[0],columns=items_list)
        def buy_combos(dif,ovs,items_amount,hand,buy_ref,items_list,rec_list): #enter RECURSIVE function
            for item in items_list:
                hand_remains = hand-buy_ref.loc[item,:]
                if  list(hand_remains>=-dif).count(False)==0: #if nothing is overdrawn -> potential for more purchases
                    if abs(sum(hand_remains[hand_remains<0]))<=dif:
                        items_amount.loc[0,item] = items_amount.loc[0,item] + 1
                        ovs = pd.concat([ovs,items_amount],ignore_index=True)
                        ovs = buy_combos(dif,ovs,items_amount,hand_remains,buy_ref,items_list,rec_list) #re-renter function
                        items_amount.loc[0,item] = items_amount.loc[0,item] - 1
            return ovs
        #self.status = 'hand_option_variants before buy_combos' ; self.display_status()
        ovs = buy_combos(dif,ovs,items_amount,hand,buy_ref,items_list,rec_list)
        #self.status = 'hand_option_variants after buy_combos' ; self.display_status()
        ovs = ovs.drop_duplicates() ; ovs = ovs.reset_index(drop=True)
        ovs_plays = ovs ; ovs_hands=pd.DataFrame(index=ovs_plays.index,columns=hand.index)
        #calculate all hands after corresponding purchases
        for i in ovs_plays.index: ovs_hands.loc[i,:] = hand.values - np.dot(ovs_plays.loc[i,:],buy_ref)
        #self.status = 'hand_option_variants end' ; self.display_status()
        return ovs_plays, ovs_hands
    def predict_hand(self,cpu):
        predicted_pick_ups = round(self.probs.loc[cpu.name,:]*(len(self.names))*float(cpu.trait.optimism))
        predicted_hand = self.cards.loc[cpu.name,:][:5]+predicted_pick_ups
        return predicted_hand.astype(int)
    def update_values(self,cpu,settlement,city):
        self.status = 'update_values' ; self.display_status()
        new_vals = self.place.byspot.loc[settlement+city,['spot_resources','spot_dots']]
        new_vals = new_vals.reset_index(drop=True)
        recs=[]; dots=[]
        for i in new_vals.index: recs += list(new_vals.spot_resources[i]); dots += list(new_vals.spot_dots[i])
        rec_names=['wood','sheep','wheat','brick','ore']
        info=pd.DataFrame(0,index=[0],columns=rec_names)
        for idx in range(len(recs)): 
            rec=recs[idx]; dot=dots[idx]
            if rec!='desert': info.loc[0,rec]+=round(dot/36,3)
        if self.trade_ins.loc[cpu.name,'wood']==3: cpu.value['3:1'] = 0 ; factor=1.5
        else: factor=1
        for resource in info.columns:
            bump=1
            if self.trade_ins.loc[cpu.name,resource] == 2: bump=2
            elif self.trade_ins.loc[cpu.name,resource] == 3: bump=1.5
            resource_prob = info.loc[0,resource]
            tto = round(pow(cpu.trait.trade_ins/factor,2)*3*resource_prob,3) #two-to-ones
            weight = round((resource_prob*(1+cpu.trait.diversity))/bump,3)
            cpu.value['2:1'+resource] += tto
            cpu.value[resource] -= weight
            if list(cpu.value[resource])[0]<0: cpu.value[resource]=0
        return
    def road_variable_settlement(self,cpu,ovs_plays_row,bottom=0):
        ovs = ovs_plays_row
        proxy= self.pos.loc[cpu.name,'proxy_spots']
        settlements = self.pos.loc[cpu.name,'settlements']
        cities = self.pos.loc[cpu.name,'cities']
        spots_blocked = self.pos.loc[cpu.name,'spots_blocked']
        biggest_score=0 #what spot/roads has the biggest score
        spot_info=[]
        if int(ovs.road)<2: spots=proxy
        else: spots=proxy+settlements+cities
        max_dist = int(ovs.road) ; good = 'no' ; feasible = ['yes',int(ovs.road)]
        while max_dist<5 and good=='no':
            for start in spots:
                #min_dist=0
                min_dist=bottom
                if proxy.count(start)!=1: min_dist=2
                a,_,c = self.place.branching(cpu, max_dist, min_dist, start)
                if len(c)>0:
                    this_score = int(c.score[c.spot==a.spot])
                    if this_score>biggest_score: biggest_score=this_score ; spot_info=a
            if len(spot_info)==0: max_dist+=1 ; feasible[0]='no'
            else: good='yes'
        return spot_info, feasible
    def update_plan(self):
        self.trade_play = pd.DataFrame() ; self.trade_hand = pd.DataFrame() ; self.trade_value = pd.DataFrame()
        for cpu in self.players.player:
            self.status = 'update_plan for '+str(cpu.name) ; self.display_status()
            play, hand = self.hand_option_variants(float(cpu.trait.long_term_thinker),self.cards.loc[cpu.name,:][:5])
            clouds = 5-len(self.pos.loc[cpu.name,:].settlements); play = play[play.settlement<=clouds]
            clouds = 4-len(self.pos.loc[cpu.name,:].cities); play = play[play.city<=clouds]
            clouds = len(self.dcards); play = play[play.dcard<=clouds]
            hand = hand.loc[play.index,:]
            hand = pd.concat([hand],ignore_index=True); play = pd.concat([play],ignore_index=True)
            rds=-1; once_more=True
            while once_more:
                rds+=1
                ovs_row=pd.DataFrame([[rds,1,0,0]],columns=list(play.columns)).loc[0,:]
                _, feasible = self.road_variable_settlement(cpu,ovs_row,0)
                if feasible[0]=='yes' or rds>=max(play.road): once_more=False
            tell=[]
            for i in play.index:
                zipz=(play.settlement[i]==0 and play.road[i]==0 and play.city[i]==0 and play.dcard[i]==0)
                ras=(play.settlement[i]>0 and play.road[i]>=rds)
                rns=(play.settlement[i]==0 and play.road[i]>=rds and play.road[i]==max(play.road))
                tell += [ras or rns or play.city[i]>0 or play.dcard[i]>0 or zipz]
            play = play.loc[tell,:]; hand = hand.loc[tell,:]
            hand = pd.concat([hand],ignore_index=True); play = pd.concat([play],ignore_index=True)
            comparison=[]
            for idx in play.index:
                comparison += [round(np.dot(np.array(play.loc[idx,:]),np.array(cpu.desire.loc[0,:])),5)]
            hand = hand.loc[np.argmax(comparison),] ; ovs_row = play.loc[np.argmax(comparison),]
            stretch=sum(hand[list(hand<0)])
            trade_play, _, b = self.run_eval(cpu,ovs_row,1)
            for i in trade_play.index:
                play = pd.DataFrame([trade_play.loc[i,:]],index=[cpu.name])
                self.trade_play = pd.concat([self.trade_play,play],ignore_index=False)
            if list(play.item) == [None]:
                hand = pd.DataFrame([[0,0,0,0,0]],index=[cpu.name],columns=['wood','sheep','wheat','brick','ore'])
            else: hand = pd.DataFrame([hand],index=[cpu.name])
            self.trade_hand = pd.concat([self.trade_hand,hand],ignore_index=False)
            value = (1-self.probs.loc[cpu.name,:])*hand
            self.trade_value = pd.concat([self.trade_value,value],ignore_index=False)
        self.display_trade()
        return
    def making_trades(self,cpu):
        self.status = 'making_trades' ; self.display_status()
        def possible_trades(self,cpu):
            #what the player needs
            ask = self.trade_value.loc[cpu.name,:]
            ask = ask[ask.values<0].sort_values()
            #what the player will trade for it
            supply = self.trade_value.loc[cpu.name,:]
            supply = supply[supply.values>0].sort_values()
            for resource in ask.index:
                for give in supply.index:
                    maybe = self.trade_value.loc[self.players.index[self.players.index!=cpu.name],resource]
                    consider = self.trade_value.loc[maybe.index[maybe>0],:]
                    for client in consider.index:
                        possible = consider.loc[client,:]
                        possible = possible[possible.values<0].sort_values()
                        for goods in possible.index:
                            if list(supply.index).count(goods)>0:
                                return pd.DataFrame({'host':cpu.name,'client':client,'take':resource,
                                             'give':goods},index=[0])
            return pd.DataFrame(columns=['host','client','take','give'])

        if sum(self.trade_hand.loc[cpu.name,:]<0) > 0: f='no'
        else: f='yes'
        while f=='no':
            deal = possible_trades(self,cpu) ; f='yes'
            if len(deal)>0:
                f = 'no' ; take= deal.loc[0,'take'] ; give=deal.loc[0,'give']
                self.cards.loc[deal.host,take] = self.cards.loc[deal.host,take] + 1
                self.cards.loc[deal.host,give] = self.cards.loc[deal.host,give] - 1
                self.cards.loc[deal.client,take] = self.cards.loc[deal.client,take] - 1
                self.cards.loc[deal.client,give] = self.cards.loc[deal.client,give] + 1
                self.status = 'made trade with '+str(list(deal.client)[0]) ; self.display_status()
                self.collect_card_data(cpu,self.status) ; self.update_plan()
        return
    def drop_cards(self):
        for name in self.cards.index[self.cards.total>7]:
            self.status = str(name)+' dropping cards' ; self.display_status()
            drop_amount = self.cards.loc[name,'total'] // 2
            vals = self.trade_value.loc[name,:]
            extras = vals[vals>0].sort_values(ascending=False)
            old_hand = self.cards.loc[name,:]
            sack = pd.DataFrame(0,index=[name],columns=self.cards.columns)
            priority = self.players.player[name].value.loc[0,:][:5]
            order = [] ; priority = priority[old_hand!=0].sort_values()
            for rec in extras.index: order += [rec]*old_hand[rec] ; priority = priority[priority.index!=rec]
            for rec in priority.index: order +=[rec]*old_hand[rec]
            for idx in range(drop_amount): rec = order[idx] ; old_hand[rec] -= 1 ; sack[rec] += 1
            self.cards.loc[name,'total'] = self.cards.loc[name,'total'] - drop_amount
            cpu=list(self.players.loc[name,:])[0]
            self.collect_card_data(cpu,self.status)
        return
    def move_robber(self,cpu):
        self.status = str(cpu.name)+' move robber' ; self.display_status()
        cnames = ['spots_to_tiles','spot_resources','spot_dots']
        robber_choose=pd.DataFrame(columns=cnames)
        for name in self.names:
            properties = self.pos.loc[name,'settlements'] + self.pos.loc[name,'cities']
            for spot in properties:
                new = self.place.byspot.loc[spot,cnames]
                for i in range(len(new[0])):
                    tile=pd.DataFrame([[new.spots_to_tiles[i],new.spot_resources[i],new.spot_dots[i]]],index=[name],columns=cnames)
                    if self.robber != int(tile.spots_to_tiles):
                        robber_choose = pd.concat([robber_choose,tile],ignore_index=False)
        #picking the person from points, then cards held
        robber_choose = robber_choose[robber_choose.spot_dots>2]
        cross = robber_choose.loc[cpu.name,'spots_to_tiles']
        if type(cross)==int: cross = [cross]
        for i in cross: robber_choose = robber_choose[robber_choose.spots_to_tiles!=i]
        sp = self.points[self.points.index!=cpu.name]
        sp = sp.loc[list(robber_choose.index.unique()),:]
        p1 = sp[sp.total==max(sp.total)].index
        if len(p1)>1:
            sc = self.cards.loc[p1,:]
            p2 = sc.index[sc.total == max(sc.total)]
            if len(p2)>1: name=random.sample(list(p2),1)[0]
            else: name=p2[0]
        else: name=p1[0]
        cpu = self.players.player[name]
        #picking the spot from the person
        sticker = robber_choose.loc[cpu.name,:]
        if list(robber_choose.index).count(cpu.name)!=1: 
            sticker = sticker[sticker.spot_dots == max(sticker.spot_dots)]
            rec = cpu.value.loc[0,sticker.spot_resources].index[np.argmin(cpu.value.loc[0,sticker.spot_resources])]
            choice = sticker[sticker.spot_resources==rec].spots_to_tiles
            if len(choice)>1: choice = random.sample(list(choice),1)[0]
            self.robber = int(choice)
        else: self.robber = sticker.spots_to_tiles
        victim = cpu.name
        self.robber_display.setposition(self.place.hcs[self.robber-1])
        self.status = 'robber on '+str(victim) ; self.display_status()
        return victim
    def draw_card(self,cpu,victim):
        self.status = str(cpu.name)+' drawing cards' ; self.display_status()
        tap =self.cards.loc[victim,:][:5]
        selection=[]
        for i in range(5): selection += [tap.index[i]]*tap[i]
        if len(selection)>0:
            pick = random.sample(selection,1)[0]
            self.cards.loc[victim,pick] = self.cards.loc[victim,pick] - 1
            self.cards.loc[victim,'total'] = self.cards.loc[victim,'total'] - 1
            self.cards.loc[cpu.name,pick] = self.cards.loc[cpu.name,pick] + 1
            self.cards.loc[cpu.name,'total'] = self.cards.loc[cpu.name,'total'] + 1
        return
    def play_dcards(self,cpu):
        if len(cpu.dcards)>0:
            for dcard in list(pd.DataFrame(cpu.dcards)[0].unique()):
                self.status = 'playing '+str(dcard)+' dcard' ; self.display_status()
                new=pd.DataFrame(data=[[dcard,len(self.rolls)]],index=[cpu.name],columns=['dcard','turn'])
                self.dcard_play = pd.concat([self.dcard_play,new],ignore_index=False)
                self.status = str(cpu.name)+'played '+str(dcard) ; self.display_status()
                self.collect_card_data(cpu,self.status)
                if dcard=='knight':
                    self.knights.loc[cpu.name,:] += 1
                    victim = self.move_robber(cpu)
                    self.draw_card(cpu,victim)
                    self.update_largest_army()
                    order=self.names.index(cpu.name)
                    ups=list(self.dcard_play.index[self.dcard_play.dcard=='knight']).count(cpu.name)
                    x=-120-self.factor_horizontal*3*4*(1+order)
                    y=self.factor_verticle*ups
                    self.vp_man.setpos(x,-290+y); self.vp_man.color(cpu.color,cpu.color)
                    self.vp_man.begin_fill(); self.vp_man.circle(3); self.vp_man.end_fill()
                elif dcard=='monopoly':
                    resource = self.trade_value.loc[cpu.name,:].index[np.argmin(self.trade_value.loc[cpu.name,:])]
                    for name in self.names:
                        if name!=cpu.name:
                                self.cards.loc[cpu.name,resource] = self.cards.loc[cpu.name,resource] + self.cards.loc[name,resource]
                                self.cards.loc[name,resource] = 0
                                self.cards.loc[name,'total'] = sum(self.cards.loc[name,:][:5])
                                self.cards.loc[cpu.name,'total'] = sum(self.cards.loc[cpu.name,:][:5])
                elif dcard=='road_builder':
                    for i in range(2):
                        roads=[]
                        ovs_row = pd.DataFrame(data=[[1,0,0,0]],columns=cpu.desire.columns)
                        rvs_info, feasible = self.road_variable_settlement(cpu,ovs_row,1)
                        if len(rvs_info)>0 and rvs_info.roads!=0: roads += rvs_info.roads[:1]   
                        settlement=[] ; city=[]
                        self.update_pos(cpu, settlement, roads, city)
                        self.place.update_running_weights(cpu, settlement, roads, city)
                        self.place.display_add_builds(cpu, settlement, city, roads)
                elif dcard=='victory_point':
                    self.points.loc[cpu.name,'victory_point'] = self.points.loc[cpu.name,'victory_point']+1
                    self.points.loc[cpu.name,'total'] = self.points.loc[cpu.name,'total']+1
                    order=self.names.index(cpu.name)
                    x=325-self.factor_horizontal*3*4*(1+order)
                    y=self.factor_verticle*self.points.victory_point[cpu.name]
                    self.vp_man.setpos(x,-290+y); self.vp_man.color(cpu.color,cpu.color)
                    self.vp_man.begin_fill(); self.vp_man.circle(3); self.vp_man.end_fill()
                elif dcard=='year_of_plenty':
                    hand = self.trade_hand.loc[cpu.name,:].sort_values()
                    resources =[]
                    if len(hand[hand<0])!=0:
                        for j in range(1,max(abs(hand[hand<0]))+1):
                                for i in hand[hand<0].index:
                                    if abs(hand[hand<0][i])>=j: resources += [i]
                    if len(resources)<2:
                        probs = self.probs.loc[cpu.name,:]
                        resources + [probs.index[np.argmin(probs)]]*(2-len(resources))
                    else: resources = resources[:2]
                    for i in resources: self.cards.loc[cpu.name,i] = self.cards.loc[cpu.name,i] + 1
                    self.cards.loc[cpu.name,'total'] = sum(self.cards.loc[cpu.name,:][:5])
                cpu.dcards.remove(dcard)
        return
    def update_largest_army(self):
        self.status = 'update_largest_army' ; self.display_status()
        has_it = self.points.index[self.points.largest_army==2]
        most = max(self.knights.knights); holder=0
        if len(has_it)==1: #someone has it already
            if int(self.knights.knights[has_it]) != most: # the holder does not have the most
                usurper = self.knights.knights.index[self.knights.knights==most]
                self.points.loc[has_it,['largest_army','total']] -= [2,2]
                self.points.loc[usurper,['largest_army','total']] += [2,2]
                holder=usurper[0]
        else: #nobody has it yet
            if most>=3: #somebody has reached 3 and takes it
                king = self.knights.knights.index[self.knights.knights==most]
                self.points.loc[king,['largest_army','total']] += [2,2]
                holder=king[0]
        if holder!=0:
            cpu=list(self.players.loc[holder,:])[0]
            self.la.clear()
            self.la.color(cpu.color)
            self.la.write('Largest Army',font=('Courier',20,'bold'),align='center')
        return
    def update_longest_road(self,cpu):
        self.status = 'update_longest_road' ; self.display_status()
        roads = self.pos.roads[cpu.name]
        tr_cons = self.place.tr_connections
        c=[0]*len(roads); k=[0]*len(roads)
        for idx in range(len(roads)):
            i=roads[idx]
            r2=[]
            for j in roads:
                if i!=j: r2+=[j]
            for j in r2:
                if tr_cons[i-1].count(j)==1: c[idx]+=1
            k[idx] = len(tr_cons[i-1])-c[idx]
        contact = pd.DataFrame({'touch':c,'open':k,'road':roads})
        edges=[]
        for idx in contact.index:
            good = (contact.touch[idx]==1 or contact.touch[idx]==2) and contact.open[idx]>0
            if good: edges+=[contact.road[idx]]
        def longr(start,spent,tally,tr_cons,roads):
            whole = tr_cons[start-1]
            good = list(set(whole) & set(roads))
            temp = pd.DataFrame(good)
            for i in spent: temp=temp[temp[0]!=i]
            good = list(temp[0])
            if len(good)>0:
                tally+=1; spent+=good ; tank=[]
                for link in good:
                    tank += [longr(link,spent,tally,tr_cons,roads)]
                tally=max(tank)
            return tally
        scores=pd.DataFrame(columns=['start','tally'])
        for start in edges: #all possible starting points
            tally=1; spent=[start]
            tally = longr(start,spent,tally,tr_cons,roads)
            new = pd.DataFrame(data=[[start,tally]],columns=['start','tally'])
            scores = pd.concat([scores,new],ignore_index=True)
        if len(scores)>0 and max(scores.tally)>=self.longest_road:
            has_it = self.points.index[self.points.longest_road==2]
            if len(has_it)==1: self.points.loc[has_it,['longest_road','total']] -= [2,2]
            self.points.loc[cpu.name,['longest_road','total']] += [2,2]
            self.longest_road=max(scores.tally)+1
            self.lr.clear()
            self.lr.color(cpu.color)
            self.lr.write('Longest Road',font=('Courier',20,'bold'),align='center')
        return
    def player_turn(self,cpu):
        player=cpu
        self.status = str(player.name)+'new turn'; self.display_status()
        self.collect_cards()
        self.update_plan()
        self.making_trades(player)
        self.display_cards()
        #chosen_buy, keep_hand = self.player_choose(player)
        chosen_buy, keep_hand = self.freedom(player)
        chosen_buy.index = [player.name]*len(chosen_buy)
        chosen_buy['turn'] = len(self.rolls)
        self.chosen_buy = pd.concat([self.chosen_buy,chosen_buy])
        self.make_move(player,chosen_buy,keep_hand)
        self.display_cards()
        self.status = 'done' ; self.display_status()
        self.collect_card_data(player,self.status)
        return
    def make_move(self,cpu,chosen_buy,keep_hand):
        self.status = str(cpu.name)+' making_moves' ; self.display_status()
        scores=[]
        for turns_away in range(2):
            scores.append(round(sum(chosen_buy.score[chosen_buy.turns_away==turns_away])/(1+turns_away),2))
        top = np.argmax(scores)
        if top==0 and scores[top]>0:
            chosen_buy = chosen_buy[chosen_buy.turns_away==top]
            self.cards.loc[cpu.name,:][:5] = list(keep_hand)
            self.cards.loc[cpu.name,'total'] = sum(self.cards.loc[cpu.name,:][:5])
            settlement=[] ; roads=[] ; city=[]
            for i in range(len(chosen_buy)):
                item=chosen_buy.item[i] ; location=chosen_buy.location[i]
                if item=='settlement': settlement.append(location)
                elif item=='city': city.append(location)
                elif item=='road': [roads.append(j) for j in location]
                elif item=='dcard' and len(self.dcards)>0: 
                    dcard = self.dcards[0] ; cpu.dcards.append(dcard) ; self.dcards.remove(dcard)
            self.update_pos(cpu, settlement, roads, city)
            self.place.update_running_weights(cpu, settlement, roads, city)
            self.place.display_add_builds(cpu, settlement, city, roads)
        return
    def display_status(self):
        if self.status_switch=='on':
            self.status_man.clear()
            self.status_man.write(self.status,font=('Courier',15, 'bold'),align='center')
            name = self.names[len(self.rolls) % len(self.names)-1]
            if self.status[:13]!='making_choice':
                new=pd.DataFrame(data=[[self.status,len(self.rolls)]],index=[name],columns=['status','turn'])
                self.status_data = pd.concat([self.status_data,new])
        return
    def display_cards(self):
        self.hist.clearstamp(self.card_stamp) ; self.z.clear() ; self.author.clear()
        colors =['dark green','green yellow','gold','chocolate','slate grey']
        w=20 ; buffer=2 ; fast=0 ; font=('Courier',15, 'bold')
        names=self.names; n=range(len(names)); cards=[] ; starting=[(-350,-300)]
        for person in n:
            cards.append(self.cards.loc[names[person],:][:5])
            b=(max(cards[person])+buffer)*w; starting.append((starting[person][0],starting[person][1]+b))
        def make_dim(w,cards,x,y):
            cord1=(y,x); cord2=(y-(w*cards),x); cord3=(y-(w*cards),x+w); cord4=(y,x+w)
            if cards==0: cord2=(y-1,x) ; cord3=(y-1,x+w)
            hist_dim = (cord1,cord2,cord3,cord4)
            return hist_dim
        form = turtle.Shape("compound")#; hist = turtle.Turtle()
        #author=turtle.Turtle() ; author.speed(fast); author.hideturtle() ; author.penup()
        #z=turtle.Turtle() ; z.speed(fast); z.hideturtle() ; z.penup()
        for person in n:
            self.author.color(self.players.player[person].color)
            self.author.setpos(starting[person][0]+(w+buffer*2)*2+w/2,starting[person][1]-w)
            for res in range(5):
                self.z.setpos(starting[person][0]+(w+buffer*2)*res+w/2,starting[person][1]+w*cards[person][res]+buffer)
                x=starting[person][0]+(w+buffer*2)*res; y=-starting[person][1]
                hist_dim = make_dim(w,cards[person][res],x,y); form.addcomponent(hist_dim,colors[res])
                self.z.write(cards[person][res],font=font,align='center')
            self.author.write(names[person],font=font,align='center')
        turtle.register_shape("hist", form); self.hist.shape("hist"); self.card_stamp = self.hist.stamp(); self.hist.hideturtle()
        return
    def calc_pvalues(self):
        # average of proportionate card contribution to each desire values
        # divided by the sum of all of that resource's tile dots on the board
        # normalized to be between 1 and 2
        for cpu in self.players.player:
            wood = (cpu.desire.road/2+cpu.desire.settlement/4+0+0)/4
            sheep = (0+cpu.desire.settlement/4+0+cpu.desire.dcard/3)/4
            wheat = (0+cpu.desire.settlement/4+cpu.desire.city*2/5+cpu.desire.dcard/3)/4
            brick = (cpu.desire.road/2+cpu.desire.settlement/4+0+0)/3
            ore = (0+0+cpu.desire.city*3/5+cpu.desire.dcard/3)/4
            recs=['wood','sheep','wheat','brick','ore']
            y=pd.DataFrame(np.array([wood,sheep,wheat,brick,ore]),index=recs,columns=['resource_vals'])
            board_val = []
            for rec in recs: board_val+=[sum(self.place.bytile.token_dots[self.place.bytile.resources==rec])]
            y.resource_vals /= board_val
            small = y.resource_vals[y.index[np.argmin(y.resource_vals)]]
            y.resource_vals-=small
            big = y.resource_vals[y.index[np.argmax(y.resource_vals)]]
            y.resource_vals/=big
            y.resource_vals+=1
            for rec in recs: cpu.value.loc[0,rec] = round(y,3).T.loc['resource_vals',rec]
        #set the differential based on the board that will be used to set ibs scores between 0 and 1
        bust = self.place.spot_weight_all.loc[:,['wood','sheep','wheat','brick','ore']]
        spot=np.argmax(np.sum(bust.T))+1
        top=0
        for cpu in self.players.player: top += int(np.dot(self.place.spot_weight_all.loc[spot,:],cpu.value.T))
        self.topspot = top/len(self.names)
        return
    def collect_card_data(self,cpu,time):
        self.display_cards()
        for cpu in self.players.player:
            time = pd.DataFrame('start',index=[cpu.name],columns=['time'])
            turn = pd.DataFrame(len(self.rolls),index=[cpu.name],columns=['turn'])
            temp = pd.concat([self.cards[self.cards.index==cpu.name],time,turn],axis=1)
            self.card_data = pd.concat([self.card_data,temp])
        return
    def display_trade(self):
        self.trade_turtle.clear()
        self.trade_turtle.setpos(-100,200)
        if len(self.trade_play)>0: play = self.trade_play[['item','location']]
        else: play=pd.DataFrame(columns=['item','location'])
        self.trade_turtle.write(round(play,2),font=('Courier',12,'bold'),align='center')
        self.trade_turtle.forward(280)
        self.trade_turtle.write(self.trade_hand,font=('Courier',12,'bold'),align='center')
        self.trade_turtle.setpos(-100,310)
        self.trade_turtle.write('What They Want',font=('Courier',17,'bold'),align='center')
        self.trade_turtle.forward(280)
        self.trade_turtle.write('Cards Needed To Get It',font=('Courier',17,'bold'),align='center')
        self.trade_turtle.setpos(-190,200)
    def play_game(self):
        self.calc_pvalues()
        self.make_placements()
        self.status_switch='on'
        time = turtle.Turtle(visible=False) ; time.penup(); time.speed(0) ; time.setposition(0,-225)
        rocket = turtle.Turtle(visible=False) ; rocket.penup()
        rocket.speed(0) ; rocket.setposition(125,-290)
        rocket.radians() ; rocket.pensize(2)
        home=pd.DataFrame([[rocket.position()]*len(self.names)],columns=self.names)
        rocket.pendown() ; rocket.setposition(325,-290)
        rocket.penup()
        for i in range(1,11):
            x=rocket.position()[0] ; y=rocket.position()[1]+self.factor_verticle+random.sample(range(5),1)[0]-2
            rocket.setposition(x,y); rocket.write(i,font=('Courier',12,'bold'),align='center')
        rocket.setposition(225,-310); rocket.write('Points Visual',font=('Courier',15,'bold'),align='center')
        rocket.setposition(-165,-310)
        rocket.write('Knights',font=('Courier',15,'bold'),align='center')
        rocket.setposition(-125,-290); rocket.pendown()
        rocket.setposition(-205,-290); rocket.penup()
        winner=False ; roll=''
        while winner==False:
            cpu = self.players.player[self.names[len(self.rolls)%len(self.names)]]
            time.clear(); time.color(cpu.color)
            time.write(str(cpu.name)+' turn #'+str(len(self.rolls)+1),font=('Courier',20, 'bold'),align='center')        
            self.player_turn(cpu)
            rocket.setposition(home[cpu.name][0])
            b = -290 + self.points.total[cpu.name] * self.factor_verticle
            a = 125 + len(self.rolls) * self.factor_horizontal
            rocket.pendown(); rocket.color(cpu.color); rocket.setposition(a,b); home[cpu.name] = [(a,b)]
            rocket.penup()
            self.display_status()
            if max(self.points.total)>=10: winner=True
        self.status = 'Game Over' ; self.display_status()
        for cpu in self.players.player:
            for title in ['settlements','roads','cities']:
                for thing in list(self.pos.loc[cpu.name,title]):
                    settlement=[]; city=[]; roads=[]
                    if title=='settlements': settlement=[thing]
                    elif title=='roads': roads=[thing]
                    elif title=='cities': city=[thing]
                    self.place.display_add_builds(cpu, settlement, city, roads)
        self.response = self.points.total - min(self.points.total)
        return
    def run_eval(self,cpu,ovs_row,turns_away):    
        cols = ['spots','vals']
        #positional info
        settlements = self.pos.settlements[cpu.name]
        cities = self.pos.cities[cpu.name]
        proxy = self.pos.proxy_spots[cpu.name]
        #buying dcards, spots and values
        card_limit = min(len(self.dcards),int(ovs_row.dcard))
        ttc = cpu.trait.ttc[0]/(1+turns_away)
        v_left = 5-list(self.dcard_play.dcard).count('victory_point')
        all_left = 37-len(self.dcard_play)
        if all_left>0:
            knight_left = 14 - list(self.dcard_play.dcard).count('knight')
            pg_v = v_left/all_left
            if self.points.largest_army[cpu.name] == 0:
                knight_away = max(self.knights.knights)+1 - self.knights.knights[cpu.name]
                pg_k = 2/knight_away*knight_left/all_left
            else: pg_k=0
        else:
            pg_v=0; pg_k=0
        pg = round(pg_v+pg_k,3)
        dcard_val = 0 + ttc + cpu.desire.dcard[0] + pg*cpu.trait.pg[0]
        if len(self.dcards)<1: dcard_val=0
        dcard_ops = pd.DataFrame(np.array([[[]]*card_limit,[dcard_val]*card_limit]).T,
                         index=['dcard']*card_limit,columns=cols)
        #building settlements and roads, spots and values
        road_limit= int(ovs_row.road) ; settlement_limit= min(5-len(settlements),ovs_row.settlement)
        existing=[]
        for i in self.pos.settlements: existing+=i
        for i in self.pos.cities: existing+=i
        settlement_ops=pd.DataFrame(columns=cols)
        road_ops=pd.DataFrame(columns=cols)
        road_set_info = pd.DataFrame(columns=['spot','roads','distance','start','score'])
        zone=(settlements+cities+proxy)
        prev_s=[] ; prev_r=[] ; keep_going=True
        while keep_going:
            a=pd.DataFrame(columns=['spot','roads','distance','start','score'])
            max_dist=road_limit; min_dist=0
            for start in zone:
                _,b,c = self.place.branching(cpu, max_dist, min_dist, start)
                if len(c)>0:
                    b.loc[:,'score'] = list(c.score.astype(float))
                    a = pd.concat([a,b],ignore_index=True)
            for ss in prev_s: a = a[a.spot!=ss]
            too_close=pd.DataFrame(columns=cols)
            for spot in prev_s:
                _, new, _ = self.place.branching(cpu,1,1,spot,raw=True)
                too_close = pd.concat([too_close,new],ignore_index=True)
            if len(too_close)>0:
                if len(too_close.spot)==1:
                    if type(too_close.spot[0])!=list: too_close = [int(too_close.spot)]
                    else: too_close = too_close.spot[0]
                else:
                    holster=[]
                    for i in too_close.spot:
                        if type(i)==int: holster+=[i]
                    too_close = list(holster)
                bad = list(set(a.spot) & set(too_close))
            else: bad=[]
            for i in bad: a = a[a.spot!=i]
            a = a.sort_values(by=['score','distance'],ascending=[False,True],ignore_index=True)
            a = a.reset_index(drop=True)
            if len(a)>0 and settlement_limit>0:
                ibs = a.score[0]
                ibs = int(ibs)/self.topspot*cpu.trait.ibs[0]
                ttc = cpu.trait.ttc[0]/(1+turns_away)
                val = ibs + ttc + cpu.desire.settlement[0] + cpu.trait.pg[0]
                temp = pd.DataFrame(data=[[a.spot[0],val]],index=['settlement'],columns=cols)
                settlement_ops = pd.concat([settlement_ops,temp],ignore_index=False)
                temp = pd.DataFrame(data=[[a.roads[0],0]],index=['road'],columns=cols)
                road_ops = pd.concat([road_ops,temp],ignore_index=False)
                prev_s += [int(a.spot[0])]
                zone+=[a.spot[0]]
                if a.roads[0]!=0:
                    prev_r += a.roads[0]
                    for r in a.roads[0]: zone += self.place.tr_to_s.spot[r]
                road_limit-=a.distance[0] ; settlement_limit-=1
            elif len(a)>0 and road_limit>0:
                ttc = cpu.trait.ttc[0]/(1+turns_away)
                road_val = (0 + ttc + cpu.desire.road[0])#*a.distance[0]
                temp = pd.DataFrame(data=[[a.roads[0],road_val]],index=['road'],columns=cols)
                road_ops = pd.concat([road_ops,temp],ignore_index=False)
                prev_s += [int(a.spot[0])]
                road_limit-=a.distance[0]
                if a.roads[0]!=0:
                    for r in a.roads[0]: zone += self.place.tr_to_s.spot[r]
                else: keep_going=False
            else: keep_going=False
            if road_limit<0: road_limit=0
            zone = list(pd.DataFrame(zone)[0].unique())
        road_ops = road_ops[road_ops.spots!=0]
        #building cities, spots and values
        valid = settlements+list(settlement_ops.spots)
        city_limit = min(4-len(cities),ovs_row.city,len(valid))
        cities=[]; vals=[]
        if city_limit<=0: spots=[]
        for spot in valid:
            ibs = np.dot(self.place.spot_weight_all.loc[spot,:],cpu.value.T)
            ibs = int(ibs)/self.topspot*cpu.trait.ibs[0]
            ttc = cpu.trait.ttc[0]/(1+turns_away)
            val = ibs + ttc + cpu.desire.city[0] + cpu.trait.pg[0]
            vals += [val]
        idx=['city']*city_limit
        if len(valid[:city_limit])>0:
            a=valid[:city_limit][0]; b=vals[:city_limit][0]; c = [[a,b]]
            city_ops = pd.DataFrame(data=c,index=idx,columns=cols)
        else:
            city_ops = pd.DataFrame(data=np.array([valid[:city_limit],vals[:city_limit]]).T,
                                    index=['city']*city_limit,columns=cols)
        city_ops = city_ops.sort_values(by=['vals'],ascending=False)
        operations = pd.concat([road_ops,settlement_ops,city_ops,dcard_ops])
        operations = operations.reset_index()
        operations = operations.rename(columns={'spots':'location','vals':'score','index':'item'})
        if len(operations)==0:
            operations = pd.DataFrame([[None,None,0,turns_away]],columns=['item','location','score','turns_away'])
        else: operations.loc[:,'turns_away'] = [turns_away]
        operations = pd.concat([operations],ignore_index=True)
        ct=city_limit - sum(operations.item=='city')
        refund_factors = pd.DataFrame([[road_limit,settlement_limit,ct]],columns=['r_lim','s_lim','c_lim'])
        score = sum(operations.score)
        return operations, refund_factors, score
    def freedom(self,cpu):
        self.status = str(cpu.name)+' making_choice' ; self.display_status()
        chosen_buy = pd.DataFrame(columns=['item','location','score','turns_away'])
        drp = [] ; drp_score=0; turns_away=0
        keep_ovs = pd.DataFrame(0,index=[0],columns=['road','settlement','city','dcard']).loc[0,:]
        keep_score=0; keep_refund=[]; keep_hand=[]
        keep_play = pd.DataFrame(columns=['item','location','score','turns_away'])
        possible_configurations = self.all_possible_hands(cpu,turns_away)
        reach = len(possible_configurations.index)
        for idx_h in possible_configurations.index:
            self.status = 'making_choice '+str(turns_away)+'-'+str(reach)+' config#'+str(idx_h+1)
            self.display_status()
            hand = possible_configurations.loc[idx_h,:]
            ovs_plays, ovs_hands = self.hand_option_variants(0,hand)
            for idx_p in ovs_plays.index:
                ovs_row = ovs_plays.loc[idx_p] ; end_hand = ovs_hands.loc[idx_p]
                if drp.count(list(ovs_row))==0:
                    drp.append(list(ovs_row))
                    operations, refund_factors, score = self.run_eval(cpu,ovs_row,turns_away)
                    if score>keep_score:
                        keep_score=score; keep_play=operations; keep_refund=refund_factors; keep_hand=end_hand; keep_ovs=ovs_row
                if list(ovs_row)==list(keep_ovs):
                    iter_hand_score = sum(self.trade_value.loc[cpu.name,:] * end_hand * -1)
                    if iter_hand_score > drp_score:
                        drp_score=iter_hand_score; keep_hand=end_hand
        chosen_buy = pd.concat([chosen_buy,keep_play],ignore_index=True)
        if keep_score==0:
            chosen_buy=pd.DataFrame({'item':[[]],'location':[[]],'score':0,'turns_away':0},index=[0])
            keep_hand = self.cards.loc[cpu.name,:]
            keep_refund = refund_factors
        present = list(self.trade_play.index).count(cpu.name)
        if present==0: tp = pd.DataFrame([[0,0,0,1]],index=[cpu.name],columns=list(self.trade_play.columns))
        elif present==1: tp = pd.DataFrame(self.trade_play.loc[cpu.name,:]).T; tp.turns_away = 1
        elif present>1: tp = pd.DataFrame(self.trade_play.loc[cpu.name,:]); tp.turns_away = 1
        chosen_buy = pd.concat([chosen_buy,tp],ignore_index=True)
        #refunds
        now = sum(chosen_buy.score[chosen_buy.turns_away==0])
        later = sum(chosen_buy.score[chosen_buy.turns_away==1])
        if now >= later and max(chosen_buy.turns_away)>0:
            keep_hand.wood += 1*int(keep_refund.s_lim) + 1*int(keep_refund.r_lim)
            keep_hand.sheep += 1*int(keep_refund.s_lim)
            keep_hand.wheat += 1*int(keep_refund.s_lim) + 2*int(keep_refund.c_lim)
            keep_hand.brick += 1*int(keep_refund.s_lim) + 1*int(keep_refund.r_lim)
            keep_hand.ore += 3*int(keep_refund.c_lim)
        #reduce road redundancy in turn comparison
        lob0 = chosen_buy[chosen_buy.turns_away==0]; lob0 = lob0[lob0.item=='road']
        lob1 = chosen_buy[chosen_buy.turns_away==1]; lob1  = lob1[lob1.item=='road']
        lob0 = lob0[lob0.score>0]; ice0=[]
        lob1 = lob1[lob1.score==0]; ice1=[]
        for i in lob0.index: ice0+=lob0.location[i]
        for i in lob1.index: ice1+=lob1.location[i]
        isec = list(set(ice0) & set(ice1))
        a = chosen_buy.turns_away==0; d = chosen_buy.turns_away==1
        b = chosen_buy.item=='road'; c = chosen_buy.score>0
        truth=[]
        for i in chosen_buy.index: truth.append(a[i] and b[i] and c[i])
        if len(isec)>0:
            #chosen_buy.loc[truth,['location','score']] = [isec,20]
            chosen_buy.loc[truth,:] = [[chosen_buy.item[truth][0], isec , 20 , chosen_buy.turns_away[truth][0]]]
            chosen_buy = pd.concat([chosen_buy[truth],chosen_buy[d]],ignore_index=True)
        #reduce dcard redundancy in turn comparison
        
        return chosen_buy, keep_hand

# Start Game

In [7]:
dcols = ['road_des','set_des','city_des','dcard_des','ti','ltt','div','ibs','ttc','response']
game_data=pd.DataFrame(columns=dcols)
for repititions in range(5):
    print('\n\n --- Game# '+str(repititions+1)+' ---\n\n')
    #make players
    susy = player('susy','red')
    sally = player('sally','white')
    betty = player('betty','blue')
    becky = player('becky','orange')
    ### make game
    superbowl = game([susy,sally,betty,becky])
    recs=['wood','sheep','wheat','brick','ore']; board_val=[]
    for rec in recs: board_val+=[sum(superbowl.place.bytile.token_dots[superbowl.place.bytile.resources==rec])]
    print(superbowl.explanatory)
    ### play game
    superbowl.play_game()
    #collect data
    recs2 = list(superbowl.alltime_cards.columns+['2'])
    apple = np.array(superbowl.response); banana=[]
    for i in apple: banana.append(sum(i<apple)+1)
    banana = pd.DataFrame(banana,index=superbowl.names)[0]
    for name in superbowl.names:
        t2=(board_val+
            list(superbowl.alltime_cards.loc[name,:])+
            list(superbowl.explanatory.loc[name,:])+
            [int(superbowl.response[name])]+
           [banana[name]])
        temp = pd.DataFrame([t2],columns=recs+recs2+dcols+['placement'])
        game_data = pd.concat([game_data,temp],ignore_index=True)
    #repititions+=1



 --- Game# 1 ---


       road_des  set_des  city_des  dcard_des    ti  ltt   div   ibs   ttc
becky      0.89     2.92      1.29       0.49  0.52  1.0  2.88  0.10  2.51
betty      0.62     2.94      1.32       0.57  3.65  4.0  3.01  0.74  0.24
sally      0.44     2.01      2.78       0.37  4.14  4.0  0.42  2.68  0.67
susy       0.86     2.94      1.16       0.06  1.43  2.0  3.00  0.57  0.14


 --- Game# 2 ---


       road_des  set_des  city_des  dcard_des    ti  ltt   div   ibs   ttc
becky      0.61     2.95      1.98       0.26  0.01  4.0  1.60  1.32  2.56
susy       0.12     1.48      2.70       0.64  0.39  3.0  1.89  4.97  0.81
betty      0.04     1.24      1.56       0.14  0.96  3.0  0.86  3.71  1.49
sally      0.39     1.56      2.06       0.77  3.47  2.0  0.15  1.89  0.97


 --- Game# 3 ---


       road_des  set_des  city_des  dcard_des    ti  ltt   div   ibs   ttc
susy       0.95     1.33      2.48       0.60  4.37  4.0  2.63  3.61  3.29
becky      0.09     1.79      2.51   

KeyboardInterrupt: 

In [ ]:
#game_data

In [ ]:
game_data.to_csv(r'C:\Users\Markymark\Desktop\game_data_new.csv', index = False, header=True)